In [ ]:
import time
import torch
from tqdm import tqdm
from Network.i2e import *
from torchvision import datasets
from torchvision.transforms import v2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
# 绘制图像
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = np.array(img)
    if len(img.shape) == 3:
        plt.imshow(np.transpose(img, (1, 2, 0)))
    else:
        plt.imshow(abs(img), cmap='gray')
    plt.xticks([])  # 去掉x轴
    plt.yticks([])  # 去掉y轴
    plt.axis('off')  # 去掉坐标轴
    plt.show()


def imgpand(img):
    img = np.array(img)
    assert len(img.shape) == 4
    zoers = np.zeros((img.shape[0], img.shape[2], img.shape[3]))
    img = np.concatenate([img, np.expand_dims(zoers, axis=1)], axis=1)
    return img

In [ ]:
transform_train = v2.Compose([
    v2.PILToTensor(),
    v2.Pad(4, padding_mode='edge'),
    v2.RandomCrop(32),
    v2.Resize((128, 128)),
    v2.RandomHorizontalFlip(0.5),
    v2.ToDtype(torch.float32, scale=True),
])

transform_test = v2.Compose([
    v2.PILToTensor(),
    v2.Resize((128, 128)),
    v2.ToDtype(torch.float32, scale=True),
])

train_dataset = datasets.CIFAR100(root='/ssd/Datasets/CIFAR100/', train=True, download=True, transform=transform_train)
val_dataset = datasets.CIFAR100(root='/ssd/Datasets/CIFAR100/', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=False,
    num_workers=8, pin_memory=True, sampler=None)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=128, shuffle=False,
    num_workers=0, pin_memory=True, sampler=None)

In [ ]:
net = I2E(ratio=0.07)
net = net.cuda()
net.eval()

In [ ]:
item = 0
totle_time = []
all_spike = []
for i, (images, target) in enumerate(tqdm(train_loader)):
    item += images.shape[0]

    temp_time = time.time()
    images = net(images.cuda())
    totle_time.append(time.time() - temp_time)

    mean_spike = images.mean((2, 3, 4))
    all_spike.append(mean_spike)

all_spike = torch.cat(all_spike, 1)
totle_time = np.array(totle_time)
print(all_spike.shape, item)
print(all_spike.mean(1), all_spike.std(1))
print(all_spike.mean(), all_spike.std())
print(totle_time.mean() / 128, totle_time.sum() / item)